<a href="https://colab.research.google.com/github/cellphone0966644265/storage-samples/blob/main/download_to_onedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Install  <img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/>

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!wget https://downloads.rclone.org/v1.52.1/rclone-v1.52.1-linux-amd64.deb
!apt install ./rclone-v1.52.1-linux-amd64.deb

# 2.Rclone </font> <font color='orange'>Authenticate One Drive </font> <img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/>


rclone.config file

1 download rclone https://rclone.org/downloads/

2 create rclone config file https://www.youtube.com/watch?v=Sp9lG_BYlSg

In [ ]:
# @title upload file rclone.config
Setup_Time_Zone = False

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#f9bd0d;\">Vui lòng upload file (rclone.conf) từ máy tính của bạn.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Tải lên tệp đã bị hủy trong khi tải tệp lên.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Cấu hình đã được thay đổi.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f9bd0d;\">Quá trình cài đặt đã hoàn tất thành công.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Tải lên tệp không thành công trong quá trình tải tệp lên.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Vui lòng chỉ tải lên một tệp mỗi lần.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Đã xảy ra lỗi trong quá trình tải tệp lên.</h2><br></center>"))

upload_conf()

# 3.Rclone<font color='yellow'>  Mount one drive

In [3]:

!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &

nohup: appending output to 'nohup.out'


#4.Paste <font color='orange'> Direct download link  <img src='https://iconarchive.com/download/i99352/dtafalonso/android-lollipop/Downloads.ico' height="100" alt="RcloneLab"/>

In [ ]:
# @title Paste link to download
file_name = input ("File name.extention")
location = input("Locattion:")
url = input("URL")
!wget -O /content/onedrive/"$location"/"$file_name" "$url"


# 5.Add <font color='orange'> magnet link/Torrent  <img src='https://libtorrent.org/img/logo-color-text.png' height="45" alt="RcloneLab"/>


In [ ]:
# @title Paste Magnet link to download
link = input ("PASTE Magnet link / Torrent link")
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': './onedrive/downloads/',
    'storage_mode': lt.storage_mode_t(2)}

print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())